In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

path = 'return-data/'
filelist = os.listdir(path) 
dfs = [pd.read_json(path+file, lines=True) for file in filelist]
toydata = pd.concat(dfs, ignore_index=True)

In [13]:
toydata.head()

,basket,returnLabel,totalAmount,transactionId,zipCode
0,"[1, 1, 0, 2, 5, 1, 2, 2]",0,264,5221131913,2321
1,"[1, 4, 4, 3, 2, 4]",1,60,4768566069,7597
2,[5],0,60,4791292369,1176
3,"[0, 4, 3]",0,333,4672834572,3087
4,"[2, 3, 1, 2, 0, 0]",0,378,8750635039,9400


In [14]:
#feature engineering basket
toydata = toydata.merge(toydata['basket'].apply(lambda x: pd.Series({'c_0': x.count(0), 'c_1': x.count(1),'c_2': x.count(2),'c_3': x.count(3),'c_4': x.count(4),'c_5': x.count(5)})), left_index=True, right_index=True)

In [15]:
toydata.drop(['basket','transactionId','zipCode'],axis=1,inplace=True)

In [16]:
def normalize(numericData):
    numericData_normalized = (numericData-numericData.min()) / (numericData.max()-numericData.min())
    return numericData_normalized 

#Normalizing
toydata['totalAmount'] = normalize(toydata['totalAmount'])
toydata['c_0'] = normalize(toydata['c_0'])
toydata['c_1'] = normalize(toydata['c_1'])
toydata['c_2'] = normalize(toydata['c_2'])
toydata['c_3'] = normalize(toydata['c_3'])
toydata['c_4'] = normalize(toydata['c_4'])
toydata['c_5'] = normalize(toydata['c_5'])

toydata.head()

,returnLabel,totalAmount,c_0,c_1,c_2,c_3,c_4,c_5
0,0,0.150030,0.142857,0.428571,0.428571,0.000,0.0,0.142857
1,1,0.029533,0.000000,0.142857,0.142857,0.125,0.3,0.000000
2,0,0.029533,0.000000,0.000000,0.000000,0.000,0.0,0.142857
3,0,0.190786,0.142857,0.000000,0.000000,0.125,0.1,0.000000
4,0,0.217366,0.285714,0.142857,0.285714,0.125,0.0,0.000000


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(toydata.drop('returnLabel',axis=1),toydata['returnLabel'], test_size=0.30,random_state=101)

In [ ]:
#Reduce dataframe to reduce calculation time
X_train = X_train

In [19]:
from scipy.spatial import distance
from collections import Counter

class KNN():
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train

    def predict(self, X_test, k):
        predictions = []
        for row in X_test:
            label = self.closest(row,k)
            predictions.append(label)
        return predictions

    def closest(self, row, k):
        distances = []
        for i in range(len(self.X_train)):
            distances.append((i,distance.euclidean(row,self.X_train[i])))
        distances = sorted(distances, key=lambda x:x[1])[0:k]
        k_indeces = []
        for i in range(k):
            k_indeces.append(distances[i][0])
        k_labels = []
        for i in range(k):
            k_labels.append(self.Y_train[k_indeces[i]])
        c = Counter(k_labels)
        return c.most_common()[0][0]

k = 5

classifier = KNN()

classifier.fit(X_train.values, Y_train.values)
predictions = classifier.predict(X_test.values, k)
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test, predictions))

KeyboardInterrupt: 